In [1]:
import sqlalchemy
DB_NAME="sqlalchemy_levelup"

# What is the SQLAlchemy ORM?

The ORM provides a way of modelling the objects of your domain and the relationships between them in Python.

It will persist these objects and their relationships for you, using a Relational Database Management System (RDBMS)

# Where does the ORM fit into SQLAlchemy?

The ORM is built on top of SQLAlchemy's lower-level **Expression Language** API. This API is for working directly with database constructs and expressions in Python (e.g. it contains a `Select` class for building `select` queries)

It is possible, and sometimes preferable, to use ONLY the Expression Language if you require more granular control or if the overhead of the ORM is undesirable.

Alternatively, you can use a combination of both the ORM and the Expression Language to add more granular controlls if and when they are needed

# A worked example - user management

Inspired by our Authentication service Thug Lyf

The domain objects are Users. Users will have attributes for their email address and password

# Defining the data model

The classes we add for our domain objects are referred to as *models* or *model classes*.

## Declarative Base Class

The ORM stores instances of your models in tables in the RDBMS (e.g. MySQL, PostgreSQL). To have the ORM look after the persistence of our domain objects, we define our model classes as subclasses of a base class from SQLAlchemy.

This base class is provided by SQLAlchemy through the `declarative_base` class factory (method that returns a Python class)

It is common to have just one `Base` class in your application. You often see it defined in the module scope, allowing other modules to import it.

In [2]:
# Create a declarative base
import sqlalchemy.ext.declarative

Base = sqlalchemy.ext.declarative.declarative_base()

## The models

In [3]:
# Note the distinction between the singular "User" classname and the "users" table name
# This is because an instance of this class is a "user" but the table can contain many users
class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

### Table objects

When we declare a subclass of the declarative base (i.e. a model), the ORM will create a `Table` object and a mapping between the model attributes (id, email, password) and the `Table` columns.

`Table` objects, as is probably clear, are the SQLAlchemy objects representing RDBMS tables

In [4]:
# The table object associated with User is accessed by the __table__ class attribute
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('email', String(length=255), table=<users>, nullable=False), Column('password', String(length=255), table=<users>, nullable=False), schema=None)

### Metadata

The declarative base has a `Metadata` object attached to it.

`Metadata` contains a registry of all the `Table` objects created when model classes are declared

We can access this registry under the `table` attribute of the `metadata` object

In [5]:
Base.metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('email', String(length=255), table=<users>, nullable=False), Column('password', String(length=255), table=<users>, nullable=False), schema=None)})

The registry `Metadata` maintains will be useful in a minute when we want to create actual RDBMS tables

# Connecting to the database

Describe engines

In [6]:
database_url = sqlalchemy.engine.url.URL(drivername='mysql', database=DB_NAME,
                                         query={'read_default_file': '~/.my.cnf'})

In [7]:
database_url

mysql:///sqlalchemy_levelup?read_default_file=~/.my.cnf

In [8]:
engine = sqlalchemy.engine.create_engine(database_url, echo=True)  # echo=True logs SQL queries to stdout

## Creating tables

Now we have an engine to provide us with database connectivity, we can use the declarative base metadata object to create database tables

In [10]:
Base.metadata.create_all(engine)

2016-12-29 15:30:16,501 INFO sqlalchemy.engine.base.Engine DESCRIBE `users`
2016-12-29 15:30:16,502 INFO sqlalchemy.engine.base.Engine ()
2016-12-29 15:30:16,510 INFO sqlalchemy.engine.base.Engine ROLLBACK
2016-12-29 15:30:16,512 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	email VARCHAR(255) NOT NULL, 
	password VARCHAR(255) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (email)
)


2016-12-29 15:30:16,514 INFO sqlalchemy.engine.base.Engine ()
2016-12-29 15:30:16,560 INFO sqlalchemy.engine.base.Engine COMMIT


# A Session with the ORM

In [27]:
# Code

# Extending the data model - Organisations

Our "customers" at GrowthIntel are businesses, not individual customers. Because of this, our Auth service groups Users into larger units, which we call Organisations.

Organisations can have many Users. Users belong to one and only one Organisation (1 to many relationship)

## Diagram

< TODO >